# Implementation of Heart Disease Classification

This notebook walks through the complete workflow of a project, featuring implementation of data preparation, visualizations, preprocessing and modeling.

---

## Table of Contents

1. [Environment Setup](#Environment-Setup)
2. [Data Ingestion](#Data-Ingestion)
3. [Data Preparation](#Data-Preparation)
   - [Handling Missing Values](#Handling-Missing-Values)



---
## Environment Setup

In [1]:
# This cell imports the core libraries used in this project.
# - pandas: for data loading and manipulation
# - numpy: for numeric processing
# - matplotlib.pyplot: for plotting
# - scikit-learn: for preprocessing, model building, and evaluation metrics

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler

from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, classification_report, roc_curve, auc,
                             RocCurveDisplay, roc_auc_score)


---
## Data Ingestion

In [2]:
try: 
    import ucimlrepo as uci
    heart_disease = uci.fetch_ucirepo(id=45) 

    X = heart_disease.data.features 
    y = heart_disease.data.targets 

    df = heart_disease.data.original
except:
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"

    df = pd.read_csv(
        url, # In case of library absence, specify path to the dataset
        header=None,
    )
    df = df.replace('?', np.nan)

# Column names based on the UCI Heart Disease (Cleveland) documentation
column_names = [
    "age",       # 0 
    "sex",       # 1 - (0 = female, 1 = male)
    "cp",        # 2 - chest pain type (1 = typical angina, 2 = atypical angina, 3 = non-anginal pain, 4 = asymptomatic)
    "trestbps",  # 3 - resting blood pressure
    "chol",      # 4 - serum cholesterol
    "fbs",       # 5 - fasting blood sugar > 120 mg/dl (0 = false; 1 = true)
    "restecg",   # 6 - resting ECG (0 = normal, 
                 #                  1 = having ST-T wave abnormality, 
                 #                  2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)
    "thalach",   # 7 - max heart rate
    "exang",     # 8 - exercise induced angina (0 = no; 1 = yes)
    "oldpeak",   # 9 - ST depression
    "slope",     # 10 - slope of ST segment (1 = upsloping, 2 = flat, 3 = downsloping)
    "ca",        # 11 - number of major vessels
    "thal",      # 12 - thalassemia (3 = normal, 6 = fixed defect, 7 = reversable defect)
    "target"        # 13 - diagnosis (0 = no disease, 1–4 = disease)
]

df = df.rename(columns=dict(zip(df.columns, column_names)))
df.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


---
## Data Preparation

At this point, we are interested in inspecting our dataset for any inconsistencies in features description or logical representation, missing values, incorrect data types, etc. 

Since dataset we are using was processed and has a good quality, some superficial examination will be enough for this step.

In [3]:
categorical = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'target']
numerical = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']


In [4]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
age,303.0,54.438944,9.038662,29.0,48.0,56.0,61.0,77.0
sex,303.0,0.679868,0.467299,0.0,0.0,1.0,1.0,1.0
cp,303.0,3.158416,0.960126,1.0,3.0,3.0,4.0,4.0
trestbps,303.0,131.689769,17.599748,94.0,120.0,130.0,140.0,200.0
chol,303.0,246.693069,51.776918,126.0,211.0,241.0,275.0,564.0
fbs,303.0,0.148515,0.356198,0.0,0.0,0.0,0.0,1.0
restecg,303.0,0.990099,0.994971,0.0,0.0,1.0,2.0,2.0
thalach,303.0,149.607261,22.875003,71.0,133.5,153.0,166.0,202.0
exang,303.0,0.326733,0.469794,0.0,0.0,0.0,1.0,1.0
oldpeak,303.0,1.039604,1.161075,0.0,0.0,0.8,1.6,6.2


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        299 non-null    float64
 12  thal      301 non-null    float64
 13  target    303 non-null    int64  
dtypes: float64(3), int64(11)
memory usage: 33.3 KB


In [6]:
# To check correctness of categorical sequence
df[categorical].agg(['min', 'max'])

,sex,cp,fbs,restecg,exang,slope,ca,thal,target
min,0,1,0,0,0,1,0.0,3.0,0
max,1,4,1,2,1,3,3.0,7.0,4


A preliminary examination of the dataset reveals several issues:
- The ca and thal columns are indicating missing or invalid values that need to be addressed.

- Several categorical features are currently treated as numerical, so their data types should be corrected to ensure proper categorical handling.

- The `thal` variable displays an irregular category structure. Since the ordering has no domain-specific meaning, it should be standardized or re-encoded.

- Because the goal is to predict the presence of heart disease rather than its severity, the multi-class categories (1–4) describing disease type can be simplified into a binary indicator. 

### Handling Missing Values

In [8]:
na = df[df.isna().any(axis=1)]

print(f'Percentage of rows with missing values: {round(len(na)/len(df) * 100,2)}%')
na

Percentage of rows with missing values: 1.98%


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
87,53,0,3,128,216,0,2,115,0,0.0,1,0.0,NaN,0
166,52,1,3,138,223,0,0,169,0,0.0,1,NaN,3.0,0
192,43,1,4,132,247,1,2,143,1,0.1,2,NaN,7.0,1
266,52,1,4,128,204,1,0,156,1,1.0,2,0.0,NaN,2
287,58,1,2,125,220,0,0,144,0,0.4,2,NaN,7.0,0
302,38,1,3,138,175,0,0,173,0,0.0,1,NaN,3.0,0


Since proportion of rows with missing values is relatively low to whole dataset, we can afford to just drop them.

In [10]:
na

dfc = df.dropna(ignore_index=True)
dfc

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,57,0,4,140,241,0,0,123,1,0.2,2,0.0,7.0,1
293,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1
294,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2
295,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
